Final Project: finding confounding variables

Idan Hermelin

Noam Biton

In [324]:
import pandas as pd

# Primary dataset
gdp_growth_df = pd.read_csv('GDPs\gdp_per_capita_growth.csv') 
#We choose to use the gdp per capita growth because it is a better indicator of the economic growth of a country than only gdp
gdp_ppp_df = pd.read_csv('GDPs\gdp_ppp_per_capita.csv')
gdp_growth_df.head()

# Auxiliary datasets
happiness_2015_df = pd.read_csv('happiness_score/2015.csv')
happiness_2016_df = pd.read_csv('happiness_score/2016.csv')
happiness_2017_df = pd.read_csv('happiness_score/2017.csv')
happiness_2018_df = pd.read_csv('happiness_score/2018.csv')
happiness_2019_df = pd.read_csv('happiness_score/2019.csv')

print("columns of the datasets:")
print(happiness_2015_df.columns)
print(happiness_2016_df.columns)
print(happiness_2017_df.columns)
print(happiness_2018_df.columns)
print(happiness_2019_df.columns)

columns of the datasets:
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Lower Confidence Interval', 'Upper Confidence Interval',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Trust (Government Corruption)', 'Generosity',
       'Dystopia Residual'],
      dtype='object')
Index(['Country', 'Happiness.Rank', 'Happiness Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
Index(['Overall rank', 'Country', 'Happiness Score', 'GDP per capita',
       'Social support', 'Health (Life

In [325]:
gdp_growth_df.rename(columns={"Country Name": "Country"}, inplace=True)

In [326]:
#change happiness columns to have the same name:
happiness_2017_df.rename(columns={"Happiness.Score": "Happiness Score", "Health..Life.Expectancy.":"Health (Life Expectancy)"}, inplace=True)
happiness_2018_df.rename(columns={"Score": "Happiness Score"}, inplace=True)
happiness_2019_df.rename(columns={"Score": "Happiness Score"}, inplace=True)
happiness_2017_df.to_csv('happiness_score/2017.csv', index=False)
happiness_2018_df.to_csv('happiness_score/2018.csv', index=False)
happiness_2019_df.to_csv('happiness_score/2019.csv', index=False)

In [327]:
#change country column to have the same name:
print(happiness_2019_df.columns)
happiness_2018_df.rename(columns={"Country or region": "Country", "Freedom to make life choices": "Freedom", "Healthy life expectancy":"Health (Life Expectancy)"}, inplace=True)
happiness_2019_df.rename( columns={"Country or region": "Country", "Freedom to make life choices": "Freedom", "Healthy..life expectancy":"Health (Life Expectancy)"}, inplace=True)
happiness_2018_df.to_csv('happiness_score/2018.csv', index=False)
happiness_2019_df.to_csv('happiness_score/2019.csv', index=False)

print(happiness_2019_df.columns)

Index(['Overall rank', 'Country', 'Happiness Score', 'GDP per capita',
       'Social support', 'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')
Index(['Overall rank', 'Country', 'Happiness Score', 'GDP per capita',
       'Social support', 'Health (Life Expectancy)', 'Freedom', 'Generosity',
       'Perceptions of corruption'],
      dtype='object')


Functions for creating dfs of GDP trends

In [328]:
def merge_happiness(gdp_growth_df, happiness_df):
    return gdp_growth_df.merge(happiness_df, on=["Country"], how="left")

In [329]:
def GDP_happiness_df(year1, year2, merged_df):
    last_10_years_df = [col for col in merged_df.columns if (col.isdigit() and int(col) >= year1 and int(col) <= year2)]
    merged_df = merged_df[["Country", "Happiness Score", "Freedom", "Health (Life Expectancy)"] + last_10_years_df]
    merged_df = merged_df.dropna()
    return merged_df

In [330]:
def calc_trend_growth(merged_df, year1, year2):
    last_10_years_cols = [col for col in merged_df.columns if (col.isdigit() and int(col) >= year1 and int(col) <= year2)]
    merged_df['GDP Growth Trend Over 10 Years'] = merged_df[last_10_years_cols].mean(axis=1, skipna=True)
    return merged_df

Create the dfs that contains the trends in the GDP:

In [331]:
# Create merged datasets for gdp per capita
merged_2015_to_2019_dfs = []
for year in range(2015, 2020):  
    happiness_df = pd.read_csv(f'happiness_score/{year}.csv')
    merged_df = merge_happiness(gdp_growth_df, happiness_df)
    merged_df = GDP_happiness_df(year-10, year, merged_df)
    merged_df = calc_trend_growth(merged_df, year-10, year)
    merged_2015_to_2019_dfs.append(merged_df)

In [332]:
merged_2015_to_2019_dfs[0].head()

,Country,Happiness Score,Freedom,Health (Life Expectancy),2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,GDP Growth Trend Over 10 Years
2,Afghanistan,3.575,0.23414,0.30335,7.207967,2.253311,11.022787,1.594215,18.515419,11.264091,-2.681061,8.974865,1.974166,-0.665291,-1.622857,5.257965
4,Angola,4.033,0.10384,0.16683,11.030783,7.582319,9.890049,7.116848,-2.808600,0.640263,-0.220843,4.706519,1.291994,1.219881,-2.468737,3.452770
5,Albania,4.959,0.35733,0.81325,6.067882,6.572923,6.787229,8.328108,4.053140,4.223084,2.821642,1.584873,1.187234,1.985388,2.516827,4.193485
8,United Arab Emirates,6.901,0.64157,0.80925,-7.020363,-4.916669,-11.345653,-10.208646,-15.151225,-5.914931,2.186344,2.257099,4.409938,4.225677,4.507818,-3.360965
9,Argentina,6.574,0.44974,0.78723,7.729489,6.955534,7.924211,3.029501,-6.854225,9.300123,4.788683,-2.145284,1.265685,-3.578581,1.629664,2.731345


In [333]:
# Normalize the GDP Growth Trend Over 10 Years column so that it is between -1 and 1:
# Use rubast scaler to normalize the data
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
for i in range(0, len(merged_2015_to_2019_dfs)):
    merged_2015_to_2019_dfs[i]['GDP Growth Trend Over 10 Years'] = scaler.fit_transform(merged_2015_to_2019_dfs[i][['GDP Growth Trend Over 10 Years']])
print(merged_2015_to_2019_dfs[0].head())


                Country  Happiness Score  Freedom  Health (Life Expectancy)  \
2           Afghanistan            3.575  0.23414                   0.30335   
4                Angola            4.033  0.10384                   0.16683   
5               Albania            4.959  0.35733                   0.81325   
8  United Arab Emirates            6.901  0.64157                   0.80925   
9             Argentina            6.574  0.44974                   0.78723   

        2005      2006       2007       2008       2009       2010      2011  \
2   7.207967  2.253311  11.022787   1.594215  18.515419  11.264091 -2.681061   
4  11.030783  7.582319   9.890049   7.116848  -2.808600   0.640263 -0.220843   
5   6.067882  6.572923   6.787229   8.328108   4.053140   4.223084  2.821642   
8  -7.020363 -4.916669 -11.345653 -10.208646 -15.151225  -5.914931  2.186344   
9   7.729489  6.955534   7.924211   3.029501  -6.854225   9.300123  4.788683   

       2012      2013      2014      2015  G

In [334]:
#check the mean value of normelized GDP growth:
for i in range(0, len(merged_2015_to_2019_dfs)):
    print(merged_2015_to_2019_dfs[i]['GDP Growth Trend Over 10 Years'].mean())

0.015250229046535372
0.0995720553521306
0.048060659961853776
0.060259684543669874
0.08367051364072979


These values are more representing the real growth in GDP because in most countries, the expected GDP growth over the last 20 years has averaged around 2–4% annually for developed economies and 5–7%

In [335]:
#!pip install statsmodels

In [339]:
import statsmodels.api as sm

def rank_confounders(df, treatment, outcome, confounders):
    rankings = []
    for confounder in confounders:
        # Model without confounder
        model1 = sm.OLS(df[outcome], sm.add_constant(df[[treatment]])).fit()
        r2_without = model1.rsquared

        # Model with confounder
        model2 = sm.OLS(df[outcome], sm.add_constant(df[[treatment, confounder]])).fit()
        r2_with = model2.rsquared

        r2_change = r2_with - r2_without
        rankings.append((confounder, r2_change))

    rankings.sort(key=lambda x: x[1], reverse=True)
    return pd.DataFrame(rankings, columns=["Confounder", "R² Change"])

In [341]:
treatment_options = ["Freedom", "Health (Life Expectancy)"]
outcome = "Happiness Score"
confounders = ["GDP Growth Trend Over 10 Years"]
for df in merged_2015_to_2019_dfs:
    print(df)
    for treatment in treatment_options:
        result_df = rank_confounders(df, treatment, outcome, confounders)
        print(result_df)
        

                  Country  Happiness Score  Freedom  Health (Life Expectancy)  \
2             Afghanistan            3.575  0.23414                   0.30335   
4                  Angola            4.033  0.10384                   0.16683   
5                 Albania            4.959  0.35733                   0.81325   
8    United Arab Emirates            6.901  0.64157                   0.80925   
9               Argentina            6.574  0.44974                   0.78723   
..                    ...              ...      ...                       ...   
252            Uzbekistan            6.003  0.65821                   0.59772   
257               Vietnam            5.360  0.59444                   0.74676   
263          South Africa            4.642  0.33207                   0.27688   
264                Zambia            5.129  0.48827                   0.29924   
265              Zimbabwe            4.610  0.25861                   0.33475   

          2005      2006   

Do the same calculation about GDP ppp:

In [ ]:
# Create merged datasets for gdp ppp per capita
# merged_2015_to_2019_ppp_dfs = []
# for year in range(2015, 2020):  
#     happiness_df = pd.read_csv(f'happiness_score/{year}.csv')
#     merged_df = merge_happiness(gdp_ppp_df, happiness_df)
#     merged_df = GDP_happiness_df(year-10, year, merged_df)
#     merged_df = calc_trend_growth(merged_df, year-10, year)
#     merged_2015_to_2019_ppp_dfs.append(merged_df)

KeyError: 'Country'